In [6]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
#from kneed import KneeLocator
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from os import listdir
from os.path import isfile, join
import os
import datetime as dt
import scipy.io
import glob
import pyresample
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import netCDF4
import datetime as dt
from netCDF4 import date2num,num2date
import matplotlib.colors as colors
import matplotlib as mpl
from netCDF4 import Dataset
import IPython.display
import cmocean
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cft
import matplotlib.dates as mdates
import plotly.express as px
from matplotlib.ticker import NullFormatter
from matplotlib import rc
#import cosima_cookbook as cc
#from mpl_toolkits.basemap import Basemap
import math
import time
import sys

def get_var_list(list_name):
    '''
    Get a list of variables associated with a pre-defined name.
    '''
    if list_name == 'ocn':
        var_list = ['sst','sss','uocn','vocn','frzmlt']
    elif list_name == 'atmo':
        var_list = ['Tair','uatm','vatm','fswdn','flwdn','snow']
    elif list_name == 'wave':
        var_list = ['aice','wave_sig_ht','peak_period','mean_wave_dir']
    elif list_name == 'ice':
        var_list = ['aice','hi','fsdrad','iage','uvel','vvel','frazil','congel']
    elif list_name == 'JRA55':
            var_list = ['airtmp']
    elif list_name == 'static':
#        var_list = ['aice','hi','hs','fsdrad','sice','iage','vlvl','vrdg']
        var_list = ['aice','hi','hs','fsdrad','iage','alvl']
    elif list_name == 'analysis':
        var_list = ['daidtt','daidtd','Tsfc','shear','divu','strength','frazil','congel','Tair','trsig','uvel','vvel','strairx','strairy','strocnx','strocny','strintx','strinty','strcorx','strcory','wave_sig_ht','peak_period','sst','frzmlt']
    else:
        var_list = [list_name]

    return var_list

def ProgressBar(Total, Progress, BarLength=20, ProgressIcon="#", BarIcon="-"):
    try:
        # You can't have a progress bar with zero or negative length.
        if BarLength <1:
            BarLength = 20
        # Use status variable for going to the next line after progress completion.
        Status = ""
        # Calcuting progress between 0 and 1 for percentage.
        Progress = float(Progress) / float(Total)
        # Doing this conditions at final progressing.
        if Progress >= 1.:
            Progress = 1
            Status = "\r\n"    # Going to the next line
        # Calculating how many places should be filled
        Block = int(round(BarLength * Progress))
        # Show this
        Bar = "[{}] {:.0f}% {}".format(ProgressIcon * Block + BarIcon * (BarLength - Block), round(Progress * 100, 0), Status)
        return Bar
    except:
        return "ERROR"


def ShowBar(Bar):
    sys.stdout.write(Bar)
    sys.stdout.flush()

def cice_netcdf_to_df(mypath, year):
    '''
    Convert a year of CICE history files into a pandas dataframe.
    '''
    os.chdir(mypath)
    file_dates = []
    print(year)
    filename =  mypath + 'iceh.' + str(year) + '-01-01.nc'

    onlyfiles = glob.glob("{path}/iceh.*{year}*".format(path=mypath, year=year))
    onlyfiles.sort()

    ds = xr.open_dataset(filename)
    LN = ds.TLON.values
    LT = ds.TLAT.values
    # Get the total number of grid points
    size = 1
    for dim in np.shape(LN): size *= dim
    aice_data = ds['aice'][0,:,:]
    mask1 = np.ma.masked_where(LT > 0.0, aice_data)
    mask2 = np.ma.masked_where(aice_data < 0.15, aice_data)
    master_mask = mask1.mask | mask2.mask
    mask = master_mask

    X_out =  np.ma.masked_array(np.empty((size,1)), mask=mask)
    
    # Loop over the files in that year
    for filecount, file in enumerate(onlyfiles):
        progressBar = "\rProgress: " + ProgressBar(len(onlyfiles), filecount+1, 20, '#', '.')
        ShowBar(progressBar)
        
        # Open the file
        filename = file
        file_dates.append(np.datetime64(file[-13:-3]))
        ds = xr.open_dataset(filename)
        
        # Get and apply masks to remove the ocean
        aice_data = ds['aice'][0,:,:]
        mask1 = np.ma.masked_where(LT > 0.0, aice_data)
        mask2 = np.ma.masked_where(aice_data < 0.15, aice_data)
        master_mask = mask1.mask | mask2.mask
        mask = master_mask
        
        # Get all the variables
        for counter, exp in enumerate(variable_list):
            data = ds[exp][0,:,:]
            data_masked = np.ma.masked_where(mask, data.values)
            data_masked_vec = data_masked.compressed()
            row_length, = data_masked_vec.shape

            if counter == 0: 
                # First file, then initialise X_temp
                X_single_file = data_masked_vec.reshape(row_length,1)
            else:
                # Else just concatenate the new data on
                X_single_file = np.concatenate([X_single_file, data_masked_vec.reshape(row_length,1)],axis=1)

        # Add on the corresponding coordinates
        LN_masked = np.ma.masked_where(mask, LN)
        LN_vec = LN_masked.compressed()
        LT_masked = np.ma.masked_where(mask, LT)
        LT_vec = LT_masked.compressed()
        X_single_file = np.concatenate([X_single_file, LN_vec.reshape(row_length,1), LT_vec.reshape(row_length,1)],axis=1)
        
        if filecount == 0: 
            # Day 1, then initialise the year file
            X_year = X_single_file
            datetime_vec =  np.tile(np.datetime64(file[-13:-3]),(row_length,1))
        else:
            X_year = np.concatenate([X_year, X_single_file],axis=0)
            datetime_vec = np.concatenate([datetime_vec, np.tile(np.datetime64(file[-13:-3]),(row_length,1))],axis=0)
    # Save as dataframe
    df_raw = pd.DataFrame(X_year, columns = variable_list+['longitude','latitude'])#,'date'])
    df_raw['date'] = datetime_vec
    print(datetime_vec.shape)
    df_raw = df_raw.dropna()
    
    return df_raw

def standardise(df_raw):
    X_temp = df_raw['aice'].values
    len_X, = X_temp.shape
    row_index = len_X
    X_train = np.zeros((row_index,1))
    for counter, exp in enumerate(variable_list):
#        progressBar = "\rProgress: " + ProgressBar(counter, len(variable_list), 20, '#', '.')
#        ShowBar(progressBar)
        
        X_temp_vec = df_raw[exp].values

        min_max_scaler = preprocessing.MinMaxScaler()
        X_temp_vec = min_max_scaler.fit_transform(X_temp_vec.reshape(row_index,1))

        # Log transformation
       # X_temp_vec = np.log(X_temp_vec+1)
       # scaler = StandardScaler()
#        X_temp_vec = min_max_scaler.fit_transform(X_temp_vec.reshape(row_index,1))
        X_train = np.concatenate([X_train, X_temp_vec],axis=1)

    temp_lon = df_raw['longitude'].to_numpy()
    temp_lat = df_raw['latitude'].to_numpy()
    X_train = np.concatenate([X_train, temp_lon.reshape(row_index,1), temp_lat.reshape(row_index,1)],axis=1)
    X_train_out = np.delete(X_train,0,1)

    df_standard = pd.DataFrame(X_train_out, columns = variable_list+['longitude','latitude'])
    df_standard['date'] = df_raw['date'].values
    #print(df_standard.describe())
    return df_standard

def kmeans_cluster(df_raw, df_standard):
    init_centroids_good = np.array([[0.76306007, 0.04041951, 0.02538844, 0.02783396, 0.05290037, 0.74416677],
                                    [0.96479608, 0.11019326, 0.10738233, 0.26478933, 0.07206991, 0.80610988],
                                    [0.96413898, 0.18858583, 0.14985236, 0.82837526, 0.10536767, 0.74644532]])
    kmeans = KMeans(
        init=init_centroids_good,
        n_clusters=3,
        n_init=1,
        max_iter=1,
        random_state=2020
    )
    
    # Take a sub-sample (same number of points for each date)
    df_temp = df_standard.drop(df_raw[df_raw.aice < 0.01].index)
    df_subsample = df_temp.groupby('date', group_keys=False).apply(lambda x: x.sample(500))
    X_train = df_subsample.iloc[:, 1:7] 

    kmeans.fit(X_train)
    kmeans.cluster_centers_ = init_centroids_good

    X_all = df_standard.iloc[:, 1:7] 
    predicted = kmeans.predict(X_all) 

    df_kmeans = df_standard
    df_kmeans['k'] = predicted
    
    print(kmeans.cluster_centers_)
    
    return df_kmeans

def map_to_netcdf(df_kmeans,savefilename,mypath,year):
    os.chdir(mypath)
    filename =  mypath + 'iceh.' + str(year) + '-01-01.nc'
    onlyfiles = glob.glob("{path}/iceh.*{year}*".format(path=mypath, year=year))
    onlyfiles.sort()
    
    filename = '/g/data/ia40/cice-dirs/runs/waves-10/history/iceh.2000-01-01.nc'
    ds = xr.open_dataset(filename)
    LN = ds.TLON.values
    LT = ds.TLAT.values
    
    unique_dates = df_kmeans['date'].unique()
    tmp1, = unique_dates.shape

    tmp2, tmp3 = LN.shape
    k_means_array = np.empty((tmp1,tmp2,tmp3))
    k_means_array[:] = np.nan

    date_length, = unique_dates.shape
    # Map the k values onto the grid
    for time_lp, date_tmp in enumerate(unique_dates):
        progressBar = "\rProgress: " + ProgressBar(date_length, time_lp+1, 20, '#', '.')
        ShowBar(progressBar)
        date_idx = df_kmeans['date'] == date_tmp

        lon = df_kmeans['longitude'][date_idx];
        lat = df_kmeans['latitude'][date_idx];
        k = df_kmeans['k'][date_idx];

        row_length = date_idx.sum()
        lon = lon.values.reshape(row_length,1)
        lat = lat.values.reshape(row_length,1)
        k = k.values.reshape(row_length,1)
        for row_lp in range(0, row_length):
            a = abs(LT-lat[row_lp])+abs(LN-lon[row_lp])
            i,j = np.unravel_index(a.argmin(),a.shape)
            k_means_array[time_lp,i,j] = k[row_lp]
  
    # Training variables for k-means
    aice_array = np.empty((tmp1,tmp2,tmp3))
    hi_array = np.empty((tmp1,tmp2,tmp3))
    hs_array = np.empty((tmp1,tmp2,tmp3))
    fsdrad_array = np.empty((tmp1,tmp2,tmp3))
    iage_array = np.empty((tmp1,tmp2,tmp3))
    alvl_array = np.empty((tmp1,tmp2,tmp3))

    # Extra variables of interest
    swh_array = np.empty((tmp1,tmp2,tmp3))
    ppd_array = np.empty((tmp1,tmp2,tmp3))


    date_length, = unique_dates.shape

    for filecount, file in enumerate(onlyfiles):
        filename = file
        progressBar = "\rProgress: " + ProgressBar(date_length, filecount+1, 20, '#', '.')
        ShowBar(progressBar)

        ds = xr.open_dataset(filename)
        LN = ds.TLON.values
        LT = ds.TLAT.values

        aice_data = ds['aice'][0,:,:]

        mask1 = np.ma.masked_where(LT > 0.0, aice_data)
        mask2 = np.ma.masked_where(aice_data < 0.15, aice_data)
        master_mask = mask1.mask | mask2.mask
        mask = master_mask

        data = ds['aice'][0,:,:]
        aice_array[filecount,:,:] = data
        aice_array[filecount,mask] = np.nan

        data = ds['hi'][0,:,:]
        hi_array[filecount,:,:] = data
        hi_array[filecount,mask] = np.nan

        data = ds['hs'][0,:,:]
        hs_array[filecount,:,:] = data
        hs_array[filecount,mask] = np.nan

        data = ds['fsdrad'][0,:,:]
        fsdrad_array[filecount,:,:] = data
        fsdrad_array[filecount,mask] = np.nan

        data = ds['iage'][0,:,:]
        iage_array[filecount,:,:] = data
        iage_array[filecount,mask] = np.nan

        data = ds['alvl'][0,:,:]
        alvl_array[filecount,:,:] = data
        alvl_array[filecount,mask] = np.nan

        # Extra variables of interest
        data = ds['wave_sig_ht'][0,:,:]
        swh_array[filecount,:,:] = data
        swh_array[filecount,mask] = np.nan

        data = ds['peak_period'][0,:,:]
        ppd_array[filecount,:,:] = data
        ppd_array[filecount,mask] = np.nan
    
    # Save to netCDF
    ds = xr.open_dataset(filename)
    HTE = ds.HTE.values
    HTN = ds.HTN.values
    tarea = ds.tarea.values
    tmask = ds.tmask.values
    
    d_vars = {"k" : (['time','nj','ni'],k_means_array,
                                  {'long_name' :"k-means_clusters",
                                   'units'     :"cluster number",
                                   '_FillValue':-2e8}),
              "aice" : (['time','nj','ni'],aice_array,
                                  {'long_name' :"Areal sea ice area proportion of cell",
                                   'units'     :"-",
                                   '_FillValue':-2e8}),
              "hi" : (['time','nj','ni'],hi_array,
                                  {'long_name' :"Grid cell mean ice thickness",
                                   'units'     :"m",
                                   '_FillValue':-2e8}),
              "hs" : (['time','nj','ni'],hs_array,
                                  {'long_name' :"Grid cell mean snow thickness",
                                   'units'     :"m",
                                   '_FillValue':-2e8}),
              "fsdrad" : (['time','nj','ni'],fsdrad_array,
                                  {'long_name' :"Representative floe radius",
                                   'units'     :"m",
                                   '_FillValue':-2e8}),
              "iage" : (['time','nj','ni'],iage_array,
                                  {'long_name' :"Sea ice age",
                                   'units'     :"m",
                                   '_FillValue':-2e8}),
              "alvl" : (['time','nj','ni'],alvl_array,
                                  {'long_name' :"Level ice area fraction",
                                   'units'     :"-",
                                   '_FillValue':-2e8}),
              "wave_sig_ht" : (['time','nj','ni'],swh_array,
                                  {'long_name' :"Signficant wave height",
                                   'units'     :"m",
                                   '_FillValue':-2e8}),
              "HTE" : (['nj','ni'],HTE,
                              {'long_name':"T cell width on East side",
                               'units'    :"m",
                               '_FillValue':-2e8}),
              "HTN" : (['nj','ni'],HTN,
                              {'long_name':"T cell width on North side",
                               'units'    :"m",
                               '_FillValue':-2e8}),
              "tarea" : (['nj','ni'],tarea,
                              {'long_name':"area of T grid cells",
                               'units'    :"m^2",
                               '_FillValue':-2e8}),
              "tmask" : (['nj','ni'],tmask,
                              {'long_name':"ocean grid mask",
                               'units'    :"Boolean",
                               '_FillValue':-2e8})}

    coords = {"LON"  : (["nj","ni"],LN,{'units':'degrees_east'}),
              "LAT"  : (["nj","ni"],LT,{'units':'degrees_north'}),
              "time" : (["time"],unique_dates)}
    attrs = {'creation_date': "2023-04-07",#datetime.now().strftime('%Y-%m-%d %H'),
             'conventions'  : "",
             'title'        : "k-means clusters for CICE-WIM standalone 1-degree data",
             'source'       : ", ",
             'comment'      : "",
             'author'       : 'Noah Day',
             'email'        : 'noah.day@adelaide.edu.au'}
    enc_dict  = {'shuffle':True,'zlib':True,'complevel':5} 
    nc_out = xr.Dataset(data_vars=d_vars,coords=coords,attrs=attrs)
    write_job = nc_out.to_netcdf(savefilename,unlimited_dims=['time'],compute=False)

In [64]:
# Read in the CICE data
var_name = 'static'
variable_list = get_var_list(var_name)
num_variables = np.size(variable_list)

savepath = '/home/566/nd0349/notebooks/'
mypath = '/g/data/ia40/cice-dirs/runs/waves-10/history/'
year = 2011

df = cice_netcdf_to_df(mypath, year)

savepath = '/g/data/ia40/sea-ice-classification/dataframes/'
savefilename = 'raw_'+str(year)+'.csv'
df.to_csv(savepath+savefilename)


2011
Progress: [####################] 100% 
(6809572, 1)


In [65]:
# Standardise the variables
var_name = 'static'
variable_list = get_var_list(var_name)
num_variables = np.size(variable_list)
mypath = '/g/data/ia40/cice-dirs/runs/waves-10/history/'
savepath = '/home/566/nd0349/notebooks/'

df_raw = pd.read_csv('/g/data/ia40/sea-ice-classification/dataframes/raw_'+str(year)+'.csv')
df_standard = standardise(df_raw)
savepath = '/g/data/ia40/sea-ice-classification/dataframes/'
savefilename = 'standard_'+str(year)+'.csv'
df_standard.to_csv(savepath+savefilename)

df_standard.describe()

,aice,hi,hs,fsdrad,iage,alvl,longitude,latitude
count,2.137572e+06,2.137572e+06,2.137572e+06,2.137572e+06,2.137572e+06,2.137572e+06,2.137572e+06,2.137572e+06
mean,9.307068e-01,1.673727e-01,1.393969e-01,6.330059e-01,8.348495e-02,7.545040e-01,2.064478e+02,-6.846028e+01
std,1.699783e-01,1.035059e-01,1.177725e-01,3.405173e-01,6.027172e-02,2.133889e-01,1.000018e+02,4.996740e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-01,-7.762990e+01
25%,9.625749e-01,8.621833e-02,4.784778e-02,3.664212e-01,3.675344e-02,6.504778e-01,1.525000e+02,-7.243740e+01
50%,9.969490e-01,1.553418e-01,1.098350e-01,7.395617e-01,7.140754e-02,8.222344e-01,2.095000e+02,-6.870660e+01
75%,9.994976e-01,2.357784e-01,2.022592e-01,9.434922e-01,1.184501e-01,9.174587e-01,3.045000e+02,-6.503162e+01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.595000e+02,-5.303830e+01


In [62]:
# Cluster
df_raw = pd.read_csv('/g/data/ia40/sea-ice-classification/dataframes/raw_'+str(year)+'.csv')
df_standard = pd.read_csv('/g/data/ia40/sea-ice-classification/dataframes/standard_'+str(year)+'.csv')
df_kmeans = kmeans_cluster(df_raw, df_standard)
savepath = '/g/data/ia40/sea-ice-classification/dataframes/'
savefilename = 'kmeans_'+str(year)+'.csv'
df_kmeans.to_csv(savepath+savefilename)


df_kmeans.groupby('k').describe()

[[0.76306007 0.04041951 0.02538844 0.02783396 0.05290037 0.74416677]
 [0.96479608 0.11019326 0.10738233 0.26478933 0.07206991 0.80610988]
 [0.96413898 0.18858583 0.14985236 0.82837526 0.10536767 0.74644532]]


Unnamed: 0                                                             \
       count          mean            std     min        25%        50%   
k                                                                         
0   260229.0  1.036736e+06  662146.865965   675.0  468618.00   952117.0   
1   555132.0  9.423372e+05  505026.692344  1388.0  514295.75   899658.5   
2  1339919.0  1.141639e+06  647948.250059     0.0  578220.50  1203130.0   

                               aice            ... longitude         \
          75%        max      count      mean  ...       75%    max   
k                                              ...                    
0  1630154.00  2155279.0   260229.0  0.723517  ...     265.5  359.5   
1  1345712.25  2155134.0   555132.0  0.983614  ...     253.5  359.5   
2  1716856.50  2155082.0  1339919.0  0.958508  ...     315.5  359.5   

    latitude                                                                \
       count       mean       std      min       25%        50%        75%   
k                                                                            
0   260229.0 -62.902863  4.094776 -77.6299 -65.03162 -62.599540 -59.922382   
1   555132.0 -64.749081  3.688464 -77.6299 -66.69558 -64.446594 -62.599540   
2  1339919.0 -70.900176  3.878079 -77.6299 -74.10892 -70.905850 -68.225560   

              
         max  
k             
0 -53.038300  
1 -53.858738  
2 -54.665924  

[3 rows x 72 columns]

In [63]:
# Map to a netCDF
savefilename='/g/data/ia40/sea-ice-classification/kmean_'+str(year)+'.nc'
mypath = '/g/data/ia40/cice-dirs/runs/waves-10/history/'
map_to_netcdf(df_kmeans,savefilename,mypath,year)

Progress: [####################] 100% 
Progress: [####################] 100% 


In [ ]:
#datetime.now().strftime('%Y-%m-%d %H')
savefilename='/g/data/ia40/sea-ice-classification/kmean_'+str(year)+'.nc'
savefilename

In [7]:
# Analysis variables

var_name = 'analysis'
variable_list = get_var_list(var_name)
num_variables = np.size(variable_list)

savepath = '/home/566/nd0349/notebooks/'
mypath = '/g/data/ia40/cice-dirs/runs/waves-10/history/'
year = 2018

df = cice_netcdf_to_df(mypath, year)
savepath = '/g/data/ia40/sea-ice-classification/dataframes/'
savefilename = 'analysis_raw_'+str(year)+'.csv'
df.to_csv(savepath+savefilename)


2018
Progress: [####################] 100% 
(6814861, 1)


In [8]:
df.describe()

,daidtt,daidtd,Tsfc,shear,divu,strength,frazil,congel,Tair,trsig,...,strintx,strinty,strcorx,strcory,wave_sig_ht,peak_period,sst,frzmlt,longitude,latitude
count,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,...,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06,2.142861e+06
mean,1.386370e+00,-1.361813e+00,-1.209987e+01,6.736480e+00,2.993467e-01,1.076691e+04,2.352717e-01,4.932120e-01,-1.093232e+01,-8.488240e+03,...,2.743997e-03,2.104725e-03,-2.631413e-03,9.051661e-05,9.599044e-02,1.206189e+01,-1.890005e+00,-2.592636e+01,2.068146e+02,-6.849773e+01
std,7.114988e+00,4.530211e+00,8.954357e+00,9.532035e+00,5.511613e+00,9.596997e+03,4.552266e-01,6.343287e-01,8.595249e+00,1.015034e+04,...,2.530699e-02,2.216662e-02,1.404996e-02,1.792450e-02,3.869908e-01,6.324199e+00,2.250972e-01,1.169549e+02,9.924809e+01,5.010849e+00
min,-8.676929e+01,-1.241699e+02,-4.357811e+01,0.000000e+00,-1.305874e+02,2.019805e-05,0.000000e+00,0.000000e+00,-3.681859e+01,-4.393299e+05,...,-4.714528e-01,-8.035051e-01,-1.460927e-01,-2.079815e-01,0.000000e+00,0.000000e+00,-1.968267e+00,-1.000000e+03,5.000000e-01,-7.762990e+01
25%,5.333240e-02,-1.443266e+00,-1.936956e+01,1.826602e+00,-4.755968e-01,3.588815e+03,1.932373e-02,1.497240e-02,-1.757928e+01,-1.192001e+04,...,-5.014659e-03,-3.748889e-03,-8.397230e-03,-6.340887e-03,3.735831e-08,1.085520e+01,-1.926586e+00,-5.877564e+00,1.555000e+02,-7.243740e+01
50%,5.347195e-01,-6.565638e-01,-1.075332e+01,4.145495e+00,2.601054e-01,9.620862e+03,1.003949e-01,2.935034e-01,-9.761654e+00,-5.596223e+03,...,1.316632e-06,1.612066e-03,-1.699846e-03,8.445031e-04,9.152009e-05,1.478338e+01,-1.911775e+00,3.574283e-01,2.105000e+02,-6.870660e+01
75%,1.353378e+00,-2.225964e-01,-3.809207e+00,7.736388e+00,1.247540e+00,1.556972e+04,2.692455e-01,7.037333e-01,-2.772018e+00,-1.606109e+03,...,7.168001e-03,1.000495e-02,2.763798e-03,8.587679e-03,1.047749e-02,1.479290e+01,-1.897585e+00,2.217344e+00,3.035000e+02,-6.503162e+01
max,1.937742e+02,1.046825e+02,6.099917e-07,1.939003e+02,1.768665e+02,3.218868e+05,1.397282e+01,7.270258e+00,1.498381e+00,0.000000e+00,...,1.001443e+00,8.877492e-01,1.287962e-01,1.221347e-01,9.245866e+00,9.999902e+02,4.755164e+00,1.381290e+02,3.595000e+02,-5.303830e+01


In [46]:
# DAFSD terms

variable_list = ['dafsd_latm','dafsd_latg','dafsd_weld','dafsd_newi','dafsd_wave']
num_variables = np.size(variable_list)

savepath = '/home/566/nd0349/notebooks/'
mypath = '/g/data/ia40/cice-dirs/runs/waves-10/history/'
year = 2018

NFSD = np.asarray([2.6884, 9.7984, 21.6721, 40.7349, 70.1407, 113.6938, 175.5771, 259.8365, 369.6202, 506.2401, 668.2091, 850.4769])
floe_binwidth = np.asarray([5.2438, 8.9763, 14.7711, 23.3545, 35.4569, 51.6493, 72.1173, 96.4015, 123.1658, 150.0741, 173.8638, 190.6719])


os.chdir(mypath)
file_dates = []
print(year)
filename =  mypath + 'iceh.' + str(year) + '-01-01.nc'

onlyfiles = glob.glob("{path}/iceh.*{year}*".format(path=mypath, year=year))
onlyfiles.sort()
print(filename)
ds = xr.open_dataset(filename)
LN = ds.TLON.values
LT = ds.TLAT.values
# Get the total number of grid points
size = 1
for dim in np.shape(LN): size *= dim
aice_data = ds['aice'][0,:,:]
mask1 = np.ma.masked_where(LT > 0.0, aice_data)
mask2 = np.ma.masked_where(aice_data < 0.15, aice_data)
master_mask = mask1.mask | mask2.mask
mask = master_mask

X_out =  np.ma.masked_array(np.empty((size,1)), mask=mask)

# Loop over the files in that year
for filecount, file in enumerate(onlyfiles):
    progressBar = "\rProgress: " + ProgressBar(len(onlyfiles), filecount+1, 20, '#', '.')
    ShowBar(progressBar)

    # Open the file
    filename = file
    file_dates.append(np.datetime64(file[-13:-3]))
    ds = xr.open_dataset(filename)

    # Get and apply masks to remove the ocean
    aice_data = ds['aice'][0,:,:]
    mask1 = np.ma.masked_where(LT > 0.0, aice_data)
    mask2 = np.ma.masked_where(aice_data < 0.15, aice_data)
    master_mask = mask1.mask | mask2.mask
    mask = master_mask

    # Get all the variables
    for counter, exp in enumerate(variable_list):
        data3d = ds[exp][0,:,:,:]
        data = np.zeros(LN.shape)
        for nf in range(0,len(floe_binwidth)):
            data += data3d[nf,:,:].values*floe_binwidth[nf]*NFSD[nf]
            
        data_masked = np.ma.masked_where(mask, data)
        data_masked_vec = data_masked.compressed()
        row_length, = data_masked_vec.shape

        if counter == 0: 
            # First file, then initialise X_temp
            X_single_file = data_masked_vec.reshape(row_length,1)
        else:
            # Else just concatenate the new data on
            X_single_file = np.concatenate([X_single_file, data_masked_vec.reshape(row_length,1)],axis=1)

    # Add on the corresponding coordinates
    LN_masked = np.ma.masked_where(mask, LN)
    LN_vec = LN_masked.compressed()
    LT_masked = np.ma.masked_where(mask, LT)
    LT_vec = LT_masked.compressed()
    X_single_file = np.concatenate([X_single_file, LN_vec.reshape(row_length,1), LT_vec.reshape(row_length,1)],axis=1)

    if filecount == 0: 
        # Day 1, then initialise the year file
        X_year = X_single_file
        datetime_vec =  np.tile(np.datetime64(file[-13:-3]),(row_length,1))
    else:
        X_year = np.concatenate([X_year, X_single_file],axis=0)
        datetime_vec = np.concatenate([datetime_vec, np.tile(np.datetime64(file[-13:-3]),(row_length,1))],axis=0)
# Save as dataframe
df_raw = pd.DataFrame(X_year, columns = variable_list+['longitude','latitude'])#,'date'])
df_raw['date'] = datetime_vec
print(datetime_vec.shape)
df_raw = df_raw.dropna()

2018
/g/data/ia40/cice-dirs/runs/waves-10/history/iceh.2018-01-01.nc
Progress: [####################] 100% 
(6814861, 1)


In [48]:
savepath = '/g/data/ia40/sea-ice-classification/dataframes/'
savefilename = 'analysis_fsd_raw_'+str(year)+'.csv'
df_raw.to_csv(savepath+savefilename)

In [42]:






np.zeros(LN.shape)
filename =  mypath + 'iceh.' + str(year) + '-01-01.nc'
print(filename)
#xr.open_dataset(filename)
#data3d[:,:,nf].values*floe_binwidth[nf]

#data += data3d[:,:,nf]*floe_binwidth[nf]
print(floe_binwidth[nf])
print(data)
print(data3d[nf,:,:].shape)
print(np.dot(data3d[:,:,nf].values, floe_binwidth[nf]))
data = data + np.dot(data3d[nf,:,:].values, floe_binwidth[nf])

/g/data/ia40/cice-dirs/runs/waves-10/history/iceh.2018-01-01.nc
5.2438
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(300, 360)
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


ValueError: operands could not be broadcast together with shapes (300,360) (12,300) 